# Redes Neuronales Convolucionales

**Instalar Theano**

* pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git


**Instalar Tensorflow y Keras**

* conda install -c conda-forge keras

## Parte 1 - Construir el modelo de CNN

In [1]:
# Importar las liobrerías y paquetes
from keras.models import Sequential #para incializar la RN con pesos aleatorios
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
# Inicializar la CNN
classifier = Sequential() #arrancamos la RN

### --- Paso 1 - Convolución

In [4]:
#- filtres=32 detectores de características(filtros) 
#- kernel_size= (3,3) son filtros de 3x3
#- input_shape es el tamaño de las imagenes (filas, columnas, canales de color)
#- activation es poner la función de activación para despertar las neuronas una vez aplicada al convolución
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), 
                      input_shape = (64, 64, 3), activation = "relu"))

### --- Paso 2 - Max Pooling

In [5]:
#-pool_size es la matriz 2x2 del Max Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [6]:
# Una segunda capa de convolución y max pooling

classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))

classifier.add(MaxPooling2D(pool_size = (2,2)))

### --- Paso 3 - Flattening

In [7]:
classifier.add(Flatten())

### --- Paso 4 - Full Connection

In [8]:
# -units es el numero de nodos que añadimos a la capa oculta(se suele poner la media entre las entradas y el nodo de salida)
# -Kernel_initializer para elegir la funcion que usaran los pesos
# -activation es la funcion de activacion para activar en la capa oculta (relu=rectificador lineal unitario)
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dense(units = 1, activation = "sigmoid"))

In [9]:
# Compilar la CNN

# -optimizer se encarga de buscar el conjunto optimo de pesos, usamos el algortimo adam
# -loss es la funcion de perdidas, la que minimiza el error
# -metrics es la metrica (la binaria de cross entropy en este caso ya que es perro o gato)

classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Parte 2 - Ajustar la CNN a las imágenes para entrenar

In [12]:
from keras.preprocessing.image import ImageDataGenerator

#lo de abajo es copiado-pegado de https://keras.io/preprocessing/image/. Con esto realizamos cambios
#aleatorios a nuestras imagenes de training para que busquen patrones, en definitva realizamos 
#transofrmaciones a nuestras imagenes. Asi la NN tendrá mas info para aprender.

train_datagen = ImageDataGenerator(
        rescale=1./255, #transforma los px's en vez de 0 a 255 de 0 a 1
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True) #volteo horizontal

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('dataset/training_set',
                                                    target_size=(64, 64),
                                                    batch_size=32, #cantidad de imagenes que pasaran por la NN antes de volver a cargar los pesos
                                                    class_mode='binary') #perros o gatos por eso binario

testing_dataset = test_datagen.flow_from_directory('dataset/test_set',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

classifier.fit_generator(training_dataset,
                        steps_per_epoch=100,
                        epochs=3,
                        validation_data=testing_dataset,
                        validation_steps=2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/3
100/100 [==============================] - 272s 3s/step - loss: 0.6006 - accuracy: 0.6744 - val_loss: 0.5569 - val_accuracy: 0.6480
Epoch 2/3
100/100 [==============================] - 279s 3s/step - loss: 0.5823 - accuracy: 0.6856 - val_loss: 0.5279 - val_accuracy: 0.6902
Epoch 3/3
100/100 [==============================] - 242s 2s/step - loss: 0.5559 - accuracy: 0.7194 - val_loss: 0.4313 - val_accuracy: 0.7371


# Parte 3 - Cómo hacer nuevas predicciones

In [21]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/nauka.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_dataset.class_indices
if result[0][0] == 1:
    prediction = 'perro'
else:
    prediction = 'gato'

print("El animal de la foto es un " + prediction)

El animal de la foto es un perro
